In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 94 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595736 sha256=64c2a8a0143bde67dd895c4d1f0afc2a54f52d1b6555b882ecaabdd741f17c5d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 10.8 MB/s 
     |████████████████████████████████| 769 kB 12.3 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
     |████████████████████████████████| 3.0 MB 40.9 MB/s 


In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
from tensorflow.keras.preprocessing.text import Tokenizer
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import bz2
from collections import Counter
import re
import nltk
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [3]:
#데이터 불러오기
data = pd.read_csv('/content/total_final_new_label_0427_ver1.csv')

In [4]:
data.head()

,Unnamed: 0,reviews,label,new_label
0,0,﻿좋다 가격 구매 수납 공간 조금 많이 작다 정도 그냥 없다 모델 그렇다 라이트 단...,0,0
1,2,별로 사진 처럼 침대 하단 시트 가다 조금 벗겨지다 기사 설치 그렇다 지원 문제 모...,0,0
2,3,설치 기사 가다 프레임 헤드 깨다 보신 건지다 말씀 해주다 속상하다 누우 신경 쓰이...,0,0
3,4,월일 설치 사용 서랍 장이 분리 버리다 서랍 장이 톱밥 뭉치다 합판 너무 약하다 같...,0,0
4,5,보통 넉넉하다 수납 공간 구매 서랍 서랍 자체 밑바닥 바퀴 달다 서랍 뚜껑 식이 손...,0,0


In [5]:
data = data.drop(['Unnamed: 0','label'], axis=1)

In [6]:
data

,reviews,new_label
0,﻿좋다 가격 구매 수납 공간 조금 많이 작다 정도 그냥 없다 모델 그렇다 라이트 단...,0
1,별로 사진 처럼 침대 하단 시트 가다 조금 벗겨지다 기사 설치 그렇다 지원 문제 모...,0
2,설치 기사 가다 프레임 헤드 깨다 보신 건지다 말씀 해주다 속상하다 누우 신경 쓰이...,0
3,월일 설치 사용 서랍 장이 분리 버리다 서랍 장이 톱밥 뭉치다 합판 너무 약하다 같...,0
4,보통 넉넉하다 수납 공간 구매 서랍 서랍 자체 밑바닥 바퀴 달다 서랍 뚜껑 식이 손...,0
...,...,...
19747,아쉽다 기다 하지만 튼튼하다 분위기,1
19748,사용 좋다,1
19749,튼튼하다 수납 공간 좋다 여전하다 만족하다,1
19750,지방 보다 배송 늦다 기사 바쁘다 그냥 두다 가시 보다 균형 잡다 먹다 지금 흔들리...,0


In [7]:
#train & test 데이터로 나누기                                                 
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)

In [8]:
# 데이터 수 확인
print(len(train_data))
print(len(test_data))

15801
3951


In [9]:
!pip install konlpy
from konlpy.tag import Okt

okt = Okt()

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 40.7 MB/s 


In [10]:
X_train = []
for sentence in tqdm(train_data['reviews']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    X_train.append(tokenized_sentence)

100%|██████████| 15801/15801 [00:57<00:00, 274.37it/s]


In [11]:
print(X_train[23])

['냄새', '너무', '심해', '고생', 'ㅠㅠ']


In [12]:
X_test = []
for sentence in tqdm(test_data['reviews']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    X_test.append(tokenized_sentence)

100%|██████████| 3951/3951 [00:06<00:00, 623.42it/s]


In [13]:
print(X_test[23])

['진짜', '쿠션', '엄청', '푹신푹신', '오다', '같다', '소파', '티비', '편안함', '주어', '너무', '마음', '들다']


In [14]:
y_train = np.array(train_data['new_label'])
y_test = np.array(test_data['new_label'])

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [16]:
print(tokenizer.word_index)

{'좋다': 1, '배송': 2, '너무': 3, '보다': 4, '같다': 5, '가격': 6, '않다': 7, '다': 8, '생각': 9, '기사': 10, '설치': 11, '받다': 12, '제품': 13, '없다': 14, '사용': 15, '구매': 16, '오다': 17, '들다': 18, '쓰다': 19, '만족하다': 20, '그냥': 21, '깔끔하다': 22, '많이': 23, '해주다': 24, '부분': 25, '튼튼하다': 26, '친절하다': 27, '괜찮다': 28, '조립': 29, '디자인': 30, '아니다': 31, '주문': 32, '냄새': 33, '소파': 34, '대비': 35, '예쁘다': 36, '그렇다': 37, '걸리다': 38, '상품': 39, '가다': 40, '맞다': 41, '정말': 42, '색상': 43, '수납': 44, '저렴하다': 45, '크다': 46, '조금': 47, '보이': 48, '쿠션': 49, '편하다': 50, '정도': 51, '서랍': 52, '사이즈': 53, '침대': 54, '식탁': 55, '쓸다': 56, '마음': 57, '성비': 58, '주다': 59, '시간': 60, '배송비': 61, '사진': 62, '아쉽다': 63, '다리': 64, '이쁘다': 65, '가구': 66, '앉다': 67, '빠르다': 68, '느낌': 69, '공간': 70, '마감': 71, '아주': 72, '많다': 73, '싸다': 74, '그리고': 75, '교환': 76, '오래': 77, '불편하다': 78, '놓다': 79, '빼다': 80, '그래도': 81, '리다': 82, '감사하다': 83, 'ㅎㅎ': 84, 'ㅠㅠ': 85, '맞추다': 86, '진짜': 87, '엄청': 88, '매트리스': 89, '드리다': 90, '고민': 91, '별로': 92, '없이': 93, '물건': 94, '보고': 95, '처음': 96, '책상': 97, '견고하다'

In [17]:
total_cnt = len(tokenizer.word_index)

In [18]:
total_cnt

9187

In [19]:
vocab_size = total_cnt + 1

In [20]:
vocab_size

9188

In [21]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [22]:
print(X_train[:3])

[[6, 387, 8, 304, 1], [680, 47, 2126, 5, 858, 728, 281, 5370, 58, 1], [1347, 44, 23, 10, 3, 27, 336]]


In [23]:
max_len = 100

In [24]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [25]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [50]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)
cnn = Conv1D(64, 5, padding='valid', activation='relu',strides=1)(embedded_sequences)
maxpooling = MaxPooling1D(pool_size=4)(cnn)
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(maxpooling)

In [51]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [52]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 24, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [53]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

In [54]:
attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

In [55]:
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [56]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [57]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

history = model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_data=(X_test, y_test), verbose=1, callbacks=[es, mc])

Epoch 1/10
494/494 [==============================] - 38s 48ms/step - loss: 0.3152 - accuracy: 0.8613 - val_loss: 0.1547 - val_accuracy: 0.9415
Epoch 2/10
494/494 [==============================] - 21s 42ms/step - loss: 0.1102 - accuracy: 0.9617 - val_loss: 0.1474 - val_accuracy: 0.9443
Epoch 3/10
494/494 [==============================] - 20s 41ms/step - loss: 0.0676 - accuracy: 0.9772 - val_loss: 0.1749 - val_accuracy: 0.9456
Epoch 4/10
494/494 [==============================] - 20s 41ms/step - loss: 0.0492 - accuracy: 0.9834 - val_loss: 0.1512 - val_accuracy: 0.9506
Epoch 4: early stopping


In [61]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

124/124 [==============================] - 2s 15ms/step - loss: 0.1512 - accuracy: 0.9506

 테스트 정확도: 0.9506


In [60]:
print(model.evaluate(X_test, y_test))

124/124 [==============================] - 2s 15ms/step - loss: 0.1512 - accuracy: 0.9506
[0.15117135643959045, 0.950645387172699]


In [38]:
X_test

array([[   0,    0,    0, ...,   21,  252,  198],
       [   0,    0,    0, ...,   26,  144,   63],
       [   0,    0,    0, ...,   26,   48,    5],
       ...,
       [   0,    0,    0, ...,   35,   26,    1],
       [   0,    0,    0, ...,   22,   36,    1],
       [   0,    0,    0, ...,  947, 3769,  240]], dtype=int32)

In [36]:
y_test

array([0, 0, 1, ..., 0, 1, 0])